In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL.ImImagePlugin import number

current_path = os.getcwd()  # 获取当前工作目录
print("当前路径:", current_path)
current_path = current_path.replace('/samples', '')
print(sys.path) 
sys.path.append(current_path) 
print(sys.path)

当前路径: /Users/yang/PycharmProjects/Option/samples
['/Users/yang/PycharmProjects/Option/samples', '/Users/yang/PycharmProjects/Option', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa']
['/Users/yang/PycharmProjects/Option/samples', '/Users/yang/PycharmProjects/Option', '/opt/anaconda3/lib/python312.zip', '/opt/anaconda3/lib/python3.12', '/opt/anaconda3/lib/python3.12/lib-dynload', '', '/opt/anaconda3/lib/python3.12/site-packages', '/opt/anaconda3/lib/python3.12/site-packages/aeosa', '/Users/yang/PycharmProjects/Option']


In [2]:
import QuantLib as ql
import timeit
from src.QlCalendar import QlCalendar

In [3]:
start_date = ql.Date(1,1,2023)
start_date

Date(1,1,2023)

# 1 calendar
# Hong Kong Market Setup
This section demonstrates how to initialize a financial market environment for Hong Kong using QuantLib.

In [4]:
hk_calendar = QlCalendar(
    calendar='HKEx',
    init_date=start_date,
    init_risk_free_rate=0.05,
    qlDayCounter = ql.Business252
)


In [5]:
# Actual start date after market initialization
real_start_date = hk_calendar.init_date
print(f"Market Initialization Date: {hk_calendar.init_date}")
print(f"Initial Risk-Free Rate: {hk_calendar.risk_free_rate.value():.4f}")

Market Initialization Date: January 3rd, 2023
Initial Risk-Free Rate: 0.0500


In [6]:
# Update the risk-free rate to simulate market changes
hk_calendar.set_risk_free_rate(0.056)
print(f"Updated Risk-Free Rate: {hk_calendar.risk_free_rate.value():.4f}")

Updated Risk-Free Rate: 0.0560


In [7]:
# Calculate a future date (5 months from the start date)
new_start_date = hk_calendar.cal_date_advance(times=5, time_unit='months')
print(f"Start Date: {real_start_date}")
print(f"Date 5 Months Later: {new_start_date}")

Start Date: January 3rd, 2023
Date 5 Months Later: November 17th, 2025


In [8]:
# Move the market's reference date to the new date
hk_calendar.set_today(new_start_date)
print(f"Current Market Date: {hk_calendar.today()}")

Current Market Date: November 17th, 2025


In [9]:
# Calculate a future date (1 year from the current market date)
one_year_later_date = hk_calendar.cal_date_advance(init_date=hk_calendar.today(), times=1, time_unit='years')
print(f"Date 1 Year Later: {one_year_later_date}")

Date 1 Year Later: November 17th, 2026


In [10]:
# Retrieve the market calendar for business day calculations
ql_calendar = hk_calendar.calendar

In [11]:
# Calculate the number of business days between two dates
business_days = ql_calendar.businessDaysBetween(hk_calendar.today(), one_year_later_date)
print(f"Business Days Between {hk_calendar.today()} and {one_year_later_date}: {business_days}")

Business Days Between November 17th, 2025 and November 17th, 2026: 253


In [12]:
# Retrieve the day counter for calculating time fractions
ql_day_counter = hk_calendar.day_counter

In [13]:
# Calculate the actual number of days between two dates
day_counts = ql_day_counter.dayCount(hk_calendar.today(), one_year_later_date)
print(f"Calendar Days Between {hk_calendar.today()} and {one_year_later_date}: {day_counts}")

Calendar Days Between November 17th, 2025 and November 17th, 2026: 253


In [14]:
# Calculate the year fraction between two dates using the specified day count convention
day_fraction = ql_day_counter.yearFraction(hk_calendar.today(), one_year_later_date)
print(f"Year Fraction Between {hk_calendar.today()} and {one_year_later_date}: {day_fraction:.4f}")

Year Fraction Between November 17th, 2025 and November 17th, 2026: 1.0040


In [15]:
# get all trading dates list - start_date and end_date
hk_calendar.get_trading_dates(ql.Date(1,1,2025), ql.Date(30,1,2025))

array([Date(2,1,2025), Date(3,1,2025), Date(6,1,2025), Date(7,1,2025),
       Date(8,1,2025), Date(9,1,2025), Date(10,1,2025), Date(13,1,2025),
       Date(14,1,2025), Date(15,1,2025), Date(16,1,2025), Date(17,1,2025),
       Date(20,1,2025), Date(21,1,2025), Date(22,1,2025), Date(23,1,2025),
       Date(24,1,2025), Date(27,1,2025)], dtype=object)

In [16]:
# get all trading dates list = start_date and length of the list
hk_calendar.get_trading_dates(ql.Date(1,1,2025), 10)

array([Date(2,1,2025), Date(3,1,2025), Date(6,1,2025), Date(7,1,2025),
       Date(8,1,2025), Date(9,1,2025), Date(10,1,2025), Date(13,1,2025),
       Date(14,1,2025), Date(15,1,2025)], dtype=object)

# Key Concepts Explained
- **Business Days**: Days when financial markets are open, excluding weekends and holidays.
- **Day Count Convention**: A system used in finance to calculate the amount of interest accruing on a financial instrument between two dates.
- **Year Fraction**: The proportion of a year represented by the time period between two dates, calculated according to a specific day count convention.

In [17]:
hk_calendar.set_today(ql.Date(1,1,2000))

In [18]:
def cal_():
    return [hk_calendar.to_next_trading_date() for _ in range(10000)]
timeit.timeit(cal_, number=1)

0.0802622499759309

In [19]:
hk_calendar.set_today(ql.Date(1,1,2000))
cal_()

[Date(4,1,2000),
 Date(5,1,2000),
 Date(6,1,2000),
 Date(7,1,2000),
 Date(10,1,2000),
 Date(11,1,2000),
 Date(12,1,2000),
 Date(13,1,2000),
 Date(14,1,2000),
 Date(17,1,2000),
 Date(18,1,2000),
 Date(19,1,2000),
 Date(20,1,2000),
 Date(21,1,2000),
 Date(24,1,2000),
 Date(25,1,2000),
 Date(26,1,2000),
 Date(27,1,2000),
 Date(28,1,2000),
 Date(31,1,2000),
 Date(1,2,2000),
 Date(2,2,2000),
 Date(3,2,2000),
 Date(4,2,2000),
 Date(7,2,2000),
 Date(8,2,2000),
 Date(9,2,2000),
 Date(10,2,2000),
 Date(11,2,2000),
 Date(14,2,2000),
 Date(15,2,2000),
 Date(16,2,2000),
 Date(17,2,2000),
 Date(18,2,2000),
 Date(21,2,2000),
 Date(22,2,2000),
 Date(23,2,2000),
 Date(24,2,2000),
 Date(25,2,2000),
 Date(28,2,2000),
 Date(29,2,2000),
 Date(1,3,2000),
 Date(2,3,2000),
 Date(3,3,2000),
 Date(6,3,2000),
 Date(7,3,2000),
 Date(8,3,2000),
 Date(9,3,2000),
 Date(10,3,2000),
 Date(13,3,2000),
 Date(14,3,2000),
 Date(15,3,2000),
 Date(16,3,2000),
 Date(17,3,2000),
 Date(20,3,2000),
 Date(21,3,2000),
 Date(22,3